<a href="https://colab.research.google.com/github/Rkk-i8/Canalicular_excision/blob/main/Canalicular_excision1(GEE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ❶ Google Drive のマウント（再マウントを避ける）
import os
from google.colab import drive

mount_path = '/content/drive'
if not os.path.ismount(mount_path):
    drive.mount(mount_path)
else:
    print(f"Google Drive is already mounted at {mount_path}")


# ❷ 必要なライブラリのインポート
import pandas as pd
import numpy as np
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.cov_struct import Exchangeable

# ❸ Excelファイルの読み込み
xlsx_path = '/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/涙小管切断術/Canalicular Occlusion.xlsx'
df_all = pd.read_excel(xlsx_path, sheet_name="all")

# ❹ 比較対象の列セット： (変数名, pre列, 比較列, 時間ラベル)
comparisons = [
    ("Fluo_5area", "Fluo_5area_pre", "Fluo_5area_3M", "3M"),
    ("Fluo_5area", "Fluo_5area_pre", "Fluo_5area_12M", "12M"),
    ("TM_mm", "TM_mm_pre", "TM_mm_3M", "3M"),
    ("TM_mm", "TM_mm_pre", "TM_mm_12M", "12M"),
]

# ❺ GEE解析 + 記述統計をループで実施
results = []
for var_name, pre_col, post_col, label in comparisons:
    df_sub = df_all[["Patient NO", pre_col, post_col]].dropna().copy()
    df_sub = df_sub.rename(columns={pre_col: "pre", post_col: "post"})

    df_long = df_sub.melt(id_vars="Patient NO", value_vars=["pre", "post"],
                          var_name="Time", value_name=var_name)
    df_long["Time_bin"] = df_long["Time"].map({"pre": 0, "post": 1})

    # GEEモデル構築
    model = GEE.from_formula(f"{var_name} ~ Time_bin", groups="Patient NO",
                             data=df_long, family=Gaussian(),
                             cov_struct=Exchangeable())
    fit = model.fit()

    # 記述統計
    pre_stats = df_long[df_long["Time"] == "pre"][var_name]
    post_stats = df_long[df_long["Time"] == "post"][var_name]
    pre_str = f"{pre_stats.median():.3f} [{pre_stats.quantile(0.25):.3f}–{pre_stats.quantile(0.75):.3f}]"
    post_str = f"{post_stats.median():.3f} [{post_stats.quantile(0.25):.3f}–{post_stats.quantile(0.75):.3f}]"

    results.append({
        "評価項目": var_name,
        "比較": f"pre vs {label}",
        "術前中央値[IQR]": pre_str,
        "術後中央値[IQR]": post_str,
        "変化量（係数）": f"{fit.params['Time_bin']:.4f}",
        "p値": f"{fit.pvalues['Time_bin']:.4f}" if fit.pvalues['Time_bin'] >= 0.001 else "<0.001"
    })

# ❻ 結果をDataFrameとして表示
df_results = pd.DataFrame(results)
df_results


Mounted at /content/drive


,評価項目,比較,術前中央値[IQR],術後中央値[IQR],変化量（係数）,p値
0,Fluo_5area,pre vs 3M,9.000 [7.000–12.000],3.000 [2.000–6.000],-4.1212,<0.001
1,Fluo_5area,pre vs 12M,9.000 [7.000–11.750],4.000 [2.250–8.500],-3.8529,<0.001
2,TM_mm,pre vs 3M,0.129 [0.099–0.154],0.295 [0.210–0.325],0.1478,<0.001
3,TM_mm,pre vs 12M,0.129 [0.099–0.154],0.265 [0.179–0.336],0.1273,<0.001
